<H1> Les word embeddings </H1>  

Les word embeddings (*plongement de mots*) correspondent à des vecteurs qui représentent les mots d’un corpus. La proximité des vecteurs dans l’espace vectoriel correspond à une proximité "sémantique". En fait les vecteurs des mots se retrouvant dans des contextes similaires dans un corpus ont tendance à se rapprocher les uns des autres. L'objectif ici n'est pas de présenter les word embeddings mais plutôt de voir comment les utiliser concrétement.  

## Installation




Avant de commencer, il est nécessaire de déjà posséder dans son environnement toutes les librairies utiles. Dans la seconde cellule nous importons toutes les librairies qui seront utiles à ce notebook. Il se peut que, lorsque vous lanciez l'éxecution de cette cellule, une soit absente. Dans ce cas il est nécessaire de l'installer. Pour cela dans la cellule suivante utiliser la commande :  

*! pip install nom_librairie*  

**Attention :** il est fortement conseillé lorsque l'une des librairies doit être installer de relancer le kernel de votre notebook.

**Remarque :** même si toutes les librairies sont importées dès le début, les librairies utiles pour des fonctions présentées au cours de ce notebook sont ré-importées de manière à indiquer d'où elles viennent et ainsi faciliter la réutilisation de la fonction dans un autre projet.

In [ ]:
# utiliser cette cellule pour installer les librairies manquantes
# pour cela il suffit de taper dans cette cellule : !pip install nom_librairie_manquante
# d'exécuter la cellule et de relancer la cellule suivante pour voir si tout se passe bien
# recommencer tant que toutes les librairies ne sont pas installées ...


#!pip install ..

# ne pas oublier de relancer le kernel du notebook

In [ ]:
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
from collections import Counter, defaultdict
import zipfile 
import base64

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns

# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit


# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


# librairies gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors # pour charger les fichier gzip bin avec KeyedVectors.load_word2vec_format
from gensim import models
from gensim.models.word2vec import Word2Vec

# librairies NLTK
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk import word_tokenize 



# librairies Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 



# fonction utilisée pour afficher les embeddings via PCA et t-SNE
def plot_reduced(model,words=None,
                 nb_words=10,
                 svd_solver='full',
                perplexity=100, 
                init='pca', 
                n_iter=4000, 
                random_state=23):
    labels = []
    tokens = []
    if words==None:
        for word in list(model.wv.vocab)[:nb_words]: 
            tokens.append(model.wv[word])
            labels.append(word)
    else:
        for word in words[:nb_words]:
            tokens.append(model.wv[word])
            labels.append(word)
        
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(14, 7))   
    fig.tight_layout(pad=3.0)
    
    # Calcul PCA et T-SNE
    pca = PCA(n_components=2,svd_solver=svd_solver)
    pca_result = pca.fit_transform(tokens)
    
    tsne_model = TSNE(n_components=2, perplexity=perplexity, init=init, n_iter=n_iter, random_state=random_state)
    tsne_result = tsne_model.fit_transform(tokens)
    
    # plot figures 
    ax1.scatter(pca_result[:, 0], pca_result[:, 1])
    ax2.scatter(tsne_result[:, 0], tsne_result[:, 1])
    for i, word in enumerate(labels):
        ax1.annotate(word, xy=(pca_result[i, 0], pca_result[i, 1]))
        ax2.annotate(word, xy=(tsne_result[i, 0], tsne_result[i, 1]))
    ax1.set_title("PCA")
    ax2.set_title("T-SNE")
    
    plt.show()


Pour pouvoir sauvegarder sur votre répertoire Google Drive, il est nécessaire de fournir une autorisation. Pour cela il suffit d'éxecuter la ligne suivante et de saisir le code donné par Google.

In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

Corriger éventuellement la ligne ci-dessous pour mettre le chemin vers un répertoire spécifique dans votre répertoire Google Drive :

In [ ]:
import sys
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/ML_FDS'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

In [ ]:
# fonctions utilities (affichage, confusion, etc.)
from MyNLPUtilities import *

## **Première utilisation des word embeddings**  



Dans cette section, nous présentons quelques méthodes associées aux word embeddings et notamment comment apprendre ces fameux mots.  Nous utiliserons pour cela la librairie Gemsin que nous retrouverons d'ailleurs dans le notebook "7 - Extraire les topics".  

Gemsim propose une implémentation des principaux algorithmes de Word2Vec (les modèles skip-gram et CBOW). Word2Vec nécessite en entrée un texte sous la forme de tokens :

In [ ]:
Alice="""Alice was beginning to get very tired of sitting by her sister on the bank, 
and of having nothing to do: once or twice she had peeped into the book her sister was reading, 
but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice 
`without pictures or conversation?' 
So she was considering in her own mind (as well as she could, for the hot day made her feel 
very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the 
trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes 
ran close by her.There was nothing so VERY remarkable in that; nor did Alice think it so 
VERY much out of the way to hear the Rabbit say to itself, 
`Oh dear! Oh dear! I shall be late!' ..(when she thought it 
over afterwards, it occurred to her that she ought to have wondered at this, 
but at the time it all seemed quite natural). 
But when the Rabbit actually TOOK A WATCH OUT OF ITS WAISTCOAT- POCKET, and looked at it, 
and then hurried on, Alice started to her feet, for it flashed across her mind that she 
had never before see a rabbit with either a waistcoat-pocket, or a watch to take out of it, 
and burning with curiosity, she ran across the field after it, and fortunately was just in 
time to see it pop down a large rabbit-hole under the hedge. 
In another moment down went Alice after it, never once considering how in the world she was to get out again.
The rabbit-hole went straight on like a tunnel for some way, and then dipped suddenly down, 
so suddenly that Alice had not a moment to think about stopping herself before she found herself 
falling down a very deep well. 
Either the well was very deep, or she fell very slowly, for she had plenty of time as she went down 
to look about her and to wonder what was going to happen next."""

# application de word_tokenize de NTLK pour transformer en token et convertir en minuscule
sentences = [word_tokenize(s) for s in Alice.lower().split('\n')]
print ('les différents tokens')
print(sentences)

L'apprentissage des words embeddings se fait simplement en appliquant Word2Vec sur les phrases transformées. Il existe de nombreux paramètres pour Word2Vec :  
1. *sg*, 0 ou 1, qui précise si le modèle est de type Skig-gram ou CBOW, sg=0 pour CBOW (valeur par défaut) et sg=1 pour Skip-gram.
1. *min_count*, entier, qui permet d'ignorer tous les tokens qui ont une fréquence absolue inférieure à ce seuil (e.g. 1, 100). 
1. *window*, entier, qui précise la distance maximale entre le mot actuel et le mot prédit dans une phrase (e.g. 2, 10). 
1. *size*, entier, qui correspond au nombre de dimensions des vecteurs de caractéristiques (e.g. 50, 200)spécifie le nombre minimal d'occurrences d'un token pour être conservé. 
1. *alpha*, réel, qui correspond au taux d'apprentissage initial (e.g. 0,01).
1. *workers*, entier, qui spécifie le nombre de threads pour l'entraînement (utilisation de multi coeurs) (e.g. 8).

De plus amples informations sur l'ensemble des paramètres est disponbible ici : https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
# Apprentissage du modèle
model = Word2Vec(sentences, min_count=1)

# Résumé du modèle appris
print(model)

Pour connaître le vocabulaire qui a été appris :

In [ ]:
# récupération du vocabulaire qui est actuellement dans l'objet model.vw
words = list(model.wv.vocab)
print(words)

Pour connaître le vecteur associé à un mot : 

In [ ]:
print(model.wv['alice'])

L'apprentissage d'un modèle peut prendre beaucoup de temps. Il est donc préférable de le sauvegarder : 

In [ ]:
# sauvegarde du modèle
model.save('model_alice.bin')

La lecture du modèle se fait alors simplement : 

In [ ]:
new_model = Word2Vec.load('model_alice.bin')
print(new_model)

Pour afficher les mots via une ACP ou T-SNE,il suffit d'utiliser la fonction définie précédemment plot_reduced : 

In [ ]:
plot_reduced(model,nb_words=174)

Pour connaître les mots les plus proches, il suffit de faire appel à la méthode *most_similar* :

In [ ]:
les_plus_proches_d_alice=model.wv.most_similar('alice',topn=20)
print ("les 20 mots les plus proches d'alice sont :")
print (les_plus_proches_d_alice)

# pour les afficher
words_to_plot = []
words_to_plot.append("alice")
for i in range(len(les_plus_proches_d_alice)):
    words_to_plot.append(les_plus_proches_d_alice[i][0])
plot_reduced(model,words_to_plot,nb_words=len(words_to_plot))

Il est également possible de calculer la similarité entre deux mots : 

In [ ]:
print ("Similarité entre alice et rabbit : ")
model.wv.similarity('alice','rabbit')

<font color=red>Exercice :</font> en vous inspirant des codes précédents, apprenez le modèle à partir de tous les textes exprimés sur notre jeu de donées d'opinion. Nommer le modèle appris : *model_reviews* pour pouvoir l'utiliser par la suite et penser à le sauvegarder (*'model_reviews.bin'*). Faire un plot des 80 premiers mots. Vous rechercherez également les 80 termes les plus proches d'"amazon" et en ferez un plot.  



In [ ]:
df_donnees=pd.read_csv("ReviewsLabelled.csv", names=['sentence','sentiment','source'], header=0,sep='\t', encoding='utf8')
print (df_donnees.head())

# récupération des phrases dans Sentences pour faciliter le traitement
Sentences=df_donnees['sentence'].copy()

<font color=blue>Solution :</font> 


In [ ]:
# il faut au préalable transformer le jeu de données en tokens. 
# Il est possible d'utiliser le moyen précédent ou bien celui décrit ci-dessous qui utilise Spacy. 
# Ensuite, il suffit d'apprendre le modèle et de sélectionner les topn=80 most_similar.


from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

# Extraction des tokens en utilisant Spacy
nlp = English()

Sentences=df_donnees['sentence'].copy()
TokensSentences=[]
for line in Sentences:
    tokens=nlp(line)
    tok_text=[token.text.lower() for token in tokens]
    TokensSentences.append(tok_text)

# Apprentissage du modèle
model_reviews = Word2Vec(TokensSentences, min_count=1)

# Résumé du modèle appris
print(model_reviews)
# sauvegarde du modèle
model_reviews.save('model_reviews.bin')

plot_reduced(model_reviews,nb_words=80)

# les plus proches d''amazon'
les_plus_proches_d_amazon=model_reviews.wv.most_similar('amazon',topn=80)
print ("les 20 mots les plus proches d'amazon sont :")
print (les_plus_proches_d_amazon)

# pour les afficher
words_to_plot = []
words_to_plot.append("amazon")
for i in range(len(les_plus_proches_d_amazon)):
    words_to_plot.append(les_plus_proches_d_amazon[i][0])
plot_reduced(model_reviews,words_to_plot,nb_words=len(words_to_plot))

## **Des exemples de word embeddings**


 

Il existe de très nombreuses listes de word embeddings disponibles à l'heure actuelle. 

**Attention :** charger des embeddings est très coûteux en terme d'espace disque et de mémoire (RAM),   aussi  Colab risque dans sa version gratuite de ne pas avoir assez de mémoire. Aussi, dans cette section, nous ne présentons l'utilisation que d'un sous ensemble de Glove de Stanford. Des indications sont données pour utiliser le jeu de données de Google qui contient 3 millions de vecteurs de mots avec une dimension de 300 (les cellules sont actuellement au format texte, il suffit de convertir les cellules et de les exécuter via CTRL-M Y). Attention ce jeu de données est très volumineux sur votre disque et occupe beaucoup de mémoire. 




Nous utilons dans cette section un sous ensemble du  modèle Glove proposé par Stanford qui a été appris à partir de Wikipedia (2014) et de la base Gigaword 5 (ensemble de News). Dans le sous ensemble les vecteurs sont de dimensions 50 et 100. Il est possible de récupérer les données originales à l'adresse suivante : http://nlp.stanford.edu/data/glove.6B.zip ou des vecteurs de dimensions 200 et 300 sont également disponibles.

In [ ]:
!wget https://www.lirmm.fr/~poncelet/Ressources/smallglove.6B.zip

In [ ]:
# dezippe le fichier glove -> 2 fichiers textes en sortie
import zipfile # pour charger des fichiers zip
getzip = zipfile.ZipFile('smallglove.6B.zip')
getzip.extractall()

**Attention :** pour ce fichier, gensim a besoin de plus d'informations, il est donc indispensable d'ajouter  deux nombres à la première ligne. Le premier indique le nombre de mots de vocabulaire et le second indique le nombre de dimensions. Par exemple, pour effectuer les transformations sur le fichier glove.6B.100d.txt, nous pouvons utiliser le code ci-dessous:

In [ ]:
!python -m gensim.scripts.glove2word2vec --input  glove.6B.100d.txt --output glove.6B.100d.w2vformat.txt
!python -m gensim.scripts.glove2word2vec --input  glove.6B.50d.txt --output glove.6B.50d.w2vformat.txt

In [ ]:
from gensim import models
model_stanford = models.KeyedVectors.load_word2vec_format("glove.6B.100d.w2vformat.txt", binary=False)

Affichage du nombre de mots et des dix premiers mots : 

In [ ]:
words_stanford = list(model_stanford.wv.vocab)
print("nombre de mots : ",len(words_stanford))
print ('les 10 premiers mots : ')
print (words_stanford[0:10])

L'exemple bien connu des embeddings :

In [ ]:
# calcul de (king - man) + woman = ?
result = model_stanford.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)


Les mots proches de queen :

In [ ]:
# les 50 mots les plus similaires à queen
queen_words=model_stanford.wv.most_similar('queen',topn=50)
# pour les afficher
words_to_plot = []
words_to_plot.append("queen")
for i in range(len(queen_words)):
    words_to_plot.append(queen_words[i][0])
plot_reduced(model_stanford,words_to_plot,nb_words=len(words_to_plot))

In [ ]:
# Les mots les plus proches de breakfast et lunch
result = model_stanford.most_similar(positive=['breakfast', 'lunch'], topn=10)
print(result)

#### **Cas des embeddings de Google**



Pour utiliser les embeddings de Google, il faut transformer les cellules suivantes en tapant sur chaque cellule de code CTRL-M Y. 
**Attention :** le jeu de données est très volumineux et pose des problèmes de mémoire sous Colab.




!wget -P . "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!gunzip GoogleNews-vectors-negative300.bin.gz


from gensim import models

model_from_google = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

Affichage du nombre de mots et des dix premiers mots : 

words_google = list(model_from_google.wv.vocab)
print("nombre de mots : ",len(words_google))
print ('les 10 premiers mots : ')
print (words_google[0:10])

L'exemple bien connu des embeddings : calcul de (king - man) + woman = ?


result = model_from_google.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

Etant donné que les embeddings ont été appris sur un corpus de données volumineux, de nombreuses corrélations apparaissent comme nous pouvons le constater :  


queen_words=model_from_google.wv.most_similar('queen',topn=50)

words_to_plot = []
words_to_plot.append("queen")
for i in range(len(queen_words)):
    words_to_plot.append(queen_words[i][0])
plot_reduced(model_from_google,words_to_plot,nb_words=len(words_to_plot))


result = model_from_google.most_similar(positive=['breakfast', 'lunch'], topn=10)
print(result)

### **Utiliser les word embeddings dans les classifiers**

  

Dans cette section, nous analysons comment utiliser les word embeddings dans les classifieurs "traditionnels", i.e. que nous avons l'habitude d'utilser jusqu'à présent. Le principe général est dans un premier temps de rechercher et de ne retenir que les mots, i.e. des embeddings, qui sont présents dans les documents. Par la suite, étant donné que les embeddings sont des vecteurs, il est nécessaire de les transformer afin de ne conserver qu'une valeur représentative pour le mot. Pour cela, il existe différentes stratégies : utilisation de la moyenne, du min ou du max.   

Nous proposons, ci-dessous, une classe de transormation qui, à partir de documents transformés en tokens, va extraire les word embeddings correspondant aux tokens des documents et prendre en fonction du paramètre passé la moyenne, le minimum ou le maximum des valeurs du vecteur pour l'affecter au mot du corpus. 

In [ ]:
from sklearn.base import TransformerMixin
class MyEmbeddingVectorizer(TransformerMixin):
  def __init__(self, model, aggregatefunction='mean'):
    self.word_vectors = model
    self.aggregation= aggregatefunction

  def fit(self, data, y=0):
    pass

  def transform(self, data):
    # determiner la dimension des vecteurs
    size_vectors = self.word_vectors.wv.vectors
    # le nombre de dimensions est dans shape [1]
    self.D = size_vectors.shape[1]
    
    # creation d'une matrice avec des zéros
    # cela permet de remplir même si le mot n'est pas trouvé
    X = np.zeros((len(data), self.D))
    
    n = 0
    for sentence in data:
      # sentence est un document transformé en tokens
      vecs = []
      m = 0
      for word in sentence:
        try:
          # lever l'exception si un mot n'est pas trouvé dans les embeddings
          vec = self.word_vectors.wv[word]
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        # transformation du vecteur en fonction de la fonction d'agrégation
        if self.aggregation=='mean':
          X[n]= vecs.mean(axis=0)
        if self.aggregation=='min':
          X[n]= vecs.min(axis=0)
        if self.aggregation=='max':
          X[n]= vecs.max(axis=0)
      n += 1
    return X

  def fit_transform(self, data,y=0):
    self.fit(data)
    return self.transform(data)

In [ ]:
# recuperation des X et y
# les phrases ont été transformées précédemment en tokens (TokenSentences)

X=TokensSentences.copy()
y=df_donnees.sentiment

# nous utilisons une approche simple juste pour tester : 
# Création du jeu d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# transformation des tokens des phrases par le vecteur de l'embeddings auquel une
# fonction d'agrégation est appliquée ('mean', 'max', 'min')
# utilisation des embeddings issus de model_stanford
vectorizer = MyEmbeddingVectorizer(model_stanford,aggregatefunction='mean')


# transformation du jeu d'apprentissage et de test
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_tranformed = vectorizer.transform(X_test)


# utilisation de SVC pour voir le résultat
clf = SVC()

clf.fit(X_train_transformed, y_train)

y_pred = clf.predict(X_test_tranformed)

# rapport de classification et matrice de confusion
MyshowAllScores(y_test,y_pred)


In [ ]:
score = 'accuracy'
seed = 7        

X=TokensSentences.copy()
y=df_donnees.sentiment

# nous utilisons une approche simple juste pour tester : 
# Création du jeu d'apprentissage et de test
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# transformation des tokens des phrases par le vecteur de l'embeddings auquel une
# fonction d'agrégation est appliquée ('mean', 'max', 'min')
# utilisation des embeddings issus de model_stanford
vectorizer = MyEmbeddingVectorizer(model_stanford,aggregatefunction='min')


# transformation du jeu d'apprentissage et de test
X_transformed = vectorizer.fit_transform(X)
#X_test_tranformed = vectorizer.transform(X_test)
#   

# cross validation en 10 fois
kfold = KFold(n_splits=10, random_state=seed)

cv_results = cross_val_score(clf, X_transformed, y, cv=kfold, scoring=score)
print (score," : %0.3f"%(cv_results.mean()))



<font color=red>Exercice :</font> tester plusieurs valeurs de fonction d'agregation afin de voir les modifications que cela apporte au classifier. Vous pourrez également utiliser les embeddings appris sur l'ensemble du jeu de données *model_reviews* et voir si cela améliore le résultat. 

<font color=blue>Solution :</font> 





In [ ]:
# il est difficile de savoir à l'avance le type de fonction d'agrégation qui donnera le meilleur résultat. 
# Il est donc nécessaire de les expérimenter. 

# Une solution existe pour éviter ce problème : utiliser Doc2Vec dont le fonctionnement repose sur Word2vec mais tient
# compte notamment des positions des mots dans les paragraphes d'un document.


Il est bien entendu possible d'appliquer un tf-idf sur les données transformées. La classe suivante permet de convertir les embeddings et de transformer la matrice en calculant le TF-IDF associé. 

In [ ]:
from sklearn.base import TransformerMixin

class MyEmbeddingTfidfVectorizer(TransformerMixin):
    def __init__(self, model,aggregatefunction='mean'):
        #self.model = model
        self.word_vectors = model
        self.word_weight = None
        self.aggregation=aggregatefunction
 
        
    def fit(self, X, y=0):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # si un mot n'a jamais été vu, il doit être a minima aussi rare que n'importe lequel des mots déjà connu 
        # l'idf est donc initialisé avec les max des idf déjà vus. 
        max_idf = max(tfidf.idf_)
        self.word_weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    def transform(self, data):
        # determiner la dimension des vecteurs
        size_vectors = self.word_vectors.wv.vectors
        # le nombre de dimensions est dans shape [1]
        self.D = size_vectors.shape[1]
    
        # creation d'une matrice avec des zéros
        # cela permet de remplir même si le mot n'est pas trouvé
        X = np.zeros((len(data), self.D))
    
        n = 0
        for sentence in data:
            # sentence est un document transformé en tokens
            vecs = []
            m = 0
            for word in sentence:
              try:
                # lever l'exception si un mot n'est pas trouvé dans les embeddings
                vec = self.word_vectors.wv[word]
                vecs.append(vec)
                m += 1
              except KeyError:
                pass
            if len(vecs) > 0:
              vecs = np.array(vecs)
              # transformation du vecteur en fonction de la fonction d'agrégation
              if self.aggregation=='mean':
                  X[n]= vecs.mean(axis=0)
              if self.aggregation=='min':
                  X[n]= vecs.min(axis=0)
              if self.aggregation=='max':
                  X[n]= vecs.max(axis=0)
            n += 1
        return X

    def fit_transform(self, data, y=0):
      self.fit(data)
      return self.transform(data)

Nous pouvons comparer un peu les approches pour voir si les embeddings permettent d'améliorer les résultats :

In [ ]:
svc_count = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("SVC", SVC())])
svc_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("SVC", SVC(kernel="linear"))])
svc_stanford_mean = Pipeline([("embeddings vectorizer", MyEmbeddingVectorizer(model_stanford,"mean")), 
                        ("SVC", SVC(kernel="linear"))])
svc_stanford_min = Pipeline([("embeddings vectorizer", MyEmbeddingVectorizer(model_stanford,"mean")), 
                        ("SVC", SVC(kernel="linear"))])
svc_stanford_tfidf_mean = Pipeline([("embeddings tfidf vectorizer", MyEmbeddingTfidfVectorizer(model_stanford,"mean")), 
                        ("SVC", SVC(kernel="linear"))])
svc_stanford_tfidf_min = Pipeline([("embeddings tfidf vectorizer", MyEmbeddingTfidfVectorizer(model_stanford,"min")), 
                        ("SVC", SVC(kernel="linear"))])

models = [("svc_count", svc_count),
          ("svc_tfidf", svc_tfidf),
          ("svc_stanford_mean",svc_stanford_mean),
          ("svc_stanford_min",svc_stanford_min),
          ("svc_stanford_tfidf_mean",svc_stanford_tfidf_mean),
          ("svc_stanford_tfidf_min",svc_stanford_tfidf_min)
             ]

In [ ]:
# Evaluation des pipelines
score = 'accuracy'
X=TokensSentences.copy()
y=df_donnees.sentiment

MyTestPipelines(models,X,y)  

<font color=red>Exercice :</font> nous avons utilisé les embeddings de dimension 100 dans les expérimentations. Essayer sur un sous-ensemble de pipeline d'appliquer les embeddings de dimension 50, le nom du fichier est : *glove.6B.50d.w2vformat.txt*.


<font color=blue>Solution :</font>

In [ ]:
from gensim import models
# il faut d'abord charger le fichier pour apprendre les mots
print ("chargement du modèle ... ")
model_stanford_short = models.KeyedVectors.load_word2vec_format("glove.6B.50d.w2vformat.txt", binary=False)

svc_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("SVC", SVC(kernel="linear"))])
svc_stanford_mean = Pipeline([("embeddings vectorizer", MyEmbeddingVectorizer(model_stanford,"mean")), 
                        ("SVC", SVC(kernel="linear"))])
svc_stanford_short_mean = Pipeline([("embeddings vectorizer", MyEmbeddingVectorizer(model_stanford_short,"mean")), 
                        ("SVC", SVC(kernel="linear"))])
svc_stanford_tfidf_mean = Pipeline([("embeddings tfidf vectorizer", MyEmbeddingTfidfVectorizer(model_stanford,"mean")), 
                        ("SVC", SVC(kernel="linear"))])
svc_stanford_short_tfidf_mean = Pipeline([("embeddings tfidf vectorizer", MyEmbeddingTfidfVectorizer(model_stanford_short,"mean")), 
                        ("SVC", SVC(kernel="linear"))])

models = [("svc_tfidf", svc_tfidf),
          ("svc_stanford_mean",svc_stanford_mean),
          ("svc_stanford_short_mean",svc_stanford_short_mean),
          ("svc_stanford_tfidf_mean",svc_stanford_tfidf_mean),
          ("svc_stanford__short_tfidf_mean",svc_stanford_short_tfidf_mean)
             ]

# test des différents pipelines
# Evaluation des pipelines
score = 'accuracy'
X=TokensSentences.copy()
y=df_donnees.sentiment

MyTestPipelines(models,X,y)  

### **Utiliser les word embeddings dans l'apprentissage profond**

 

Les librairies comme Pytorch ou Keras proposent de pouvoir facilement manipuler les embeddings via une couche spécifique. Par la suite nous nous focaliserons plus particulièrement sur Keras.  

La couche Embedding peut être initialisée avec des poids aléatoires et dans ce cas le modèle pourra apprendre les embeddings pour tous les mots de l'ensemble de données d'apprentissage avec une approche assez similaire à word2vec. Elle peut également utiliser des embeddings appris avec un autre modèle. Nous présentons comment réaliser ces opérations dans cette section. 

Nous verrons que le principe général est assez simple et que dans un premier temps il est nécessaire de transformer les données d'entrées afin qu'elles puissent être lues par le modèle. De manière à illustrer le fonctionnement considérons l'exemple suivant : 


In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# documents
mesdocs = ["The sky is really blue today",
           "Bananas are normally yellow",
          "The nice dog jumps in the air",
          "The forest is all green",
           "The cat is purring",
           "The car drives quietly on the road",
            "The red of this car is beautiful",
           "Embeddings can help classify better"]


# les labels des classes sont positionnés à 1 si le mot contient une couleur
labels = array([1,1,0,1,0,0,1,0])

Keras propose sa propre fonction de tokenisation qui permet de faire de la tokenisation directement en effectuant certains pretraitements. Il est donc possible de transformer les documents en token et d'obtenir en même temps le vocabulaire associé ainsi qu'un index sur chaque mot de ce vocabulaire :

In [ ]:
mytokens = Tokenizer()
mytokens.fit_on_texts(mesdocs)
print ("affichage des tokens",mytokens.word_index)

# attention l'index 0 est réservé pour le padding (voir après)
# la taille du vocabulaire doit donc toujours être supérieure à 1 (d'où le +1)
vocab_size = len(mytokens.word_index) + 1
print ("vocab_size",vocab_size)

Il est maintenant nécessaire de transformer les tokens des documents en entier :

In [ ]:
# pour pouvoir traiter les documents, il est nécessaire de les encoder en entier 
# la méthode texts_to_sequences permet de transformer les documents avec l'index associé
encoded_docs = mytokens.texts_to_sequences(mesdocs)
print ("docs",mesdocs)
print("encoded_docs",encoded_docs)

Comme nous pouvons le constater les différents documents vont avoir une taille différente. Il est donc nécessaire de les homogénéiser. Pour cela il suffit de spécifier la taille désirée et de compléter le vecteur avec des 0 : padding.

**Remarque :** précédemment nous avons vu qu'il fallait ajouter un 1 pour la taille du vocabulaire. Ce 1 permet de prendre en compte que la valeur 0 ajoutée est le premier élément de l'index. 

In [ ]:
MAX_LENGTH = 10
padded_docs = pad_sequences(encoded_docs, maxlen=MAX_LENGTH, padding='post')
print("padded docs",padded_docs)

#### **Apprentissage des word embeddings**




Dans cette sous section, nous présentons comment utiliser Keras pour apprendre les word embeddings.  


La couche Embedding nécessite trois paramètres principaux : 
1. *input_dim* correspond à la taille du vocabulaire.
1. *output_dim* correspond au nombre de dimensions du vecteur pour chaque mot.
1. *input_length* correspond à la longueur des séquences d'entrée (elle doit bien sûr être compatible avec la couche d'entrée du modèle).


L'exemple ci-dessous illustre la création d'un modèle Keras :

In [ ]:
from keras.models import Model
from keras.layers import Dense , Flatten ,Embedding,Input

DIMENSION=100
# Definition du modèle pour apprendre les embeddings

# spécification du format des données d'entrées
input=Input(shape=(len(mesdocs),MAX_LENGTH),dtype='float64')
mesdocs_input=Input(shape=(MAX_LENGTH,),dtype='float64')  

# ajout de la couche des embeddings
mesdocs_embedding=Embedding(input_dim=vocab_size,output_dim=DIMENSION,input_length=MAX_LENGTH)(mesdocs_input)
mesdocs_vec=Flatten()(mesdocs_embedding) # flatten

# creation du modèle KERAS en combinant les couches précédentes
mesdocs_embed_model =Model([mesdocs_input],mesdocs_vec) 
# compilation du modèle
mesdocs_embed_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 



L'instanciation se fait en appliquant un predict sur les phrases transformées en tokens et sur lesquelles un padding a été appliqué : 

In [ ]:
# Instanciation du modèle pour déterminer les embeddings des tokens 
mesdocs_embeddings=mesdocs_embed_model.predict(padded_docs) # finally getting the embeddings.
print("Taille des embeddings appris : ",mesdocs_embeddings.shape)


Pour connaître les embeddings appris, il suffit d'accéder à la couche Embedding : 


In [ ]:
# récupération des poids qui avaient été initialisés aléatoirement et qui correspondent maintenant aux embeddings
mesdocs_layer_embeddings = mesdocs_embed_model.get_weights()[0]

# création d'un dictionnaire qui retourne pour chaque numéro d'index le token associé
mesdocs_words_embeddings = {w:mesdocs_layer_embeddings[idx] for w, idx in mytokens.word_index.items()}

# les embeddings associés à blue
print ("les embeddings de 'blue' sont :")
print (mesdocs_words_embeddings['blue'])



La fonction suivante permet de récupérer les embeddings qui sont appris sur un modèle. Le principe est similaire à ce que nous avons vu précédemment : parcours des embeddings pour ne retenir que les mots qui apparaissent dans le corpus d'apprentissage. Par contre ici nous ne transformons pas le vecteur par une fonction d'agrégation.   


Nous utilisons ici les embeddings issus de stanford (dimension 100). 

In [ ]:
def get_embedding_matrix(embeddings,words,vocabulary_size,dimension):
  # Preparation de la matrice des embeddings embedding matrix
  embedding_matrix = np.zeros((vocabulary_size, dimension))
  for word, i in words.word_index.items():
    try: # lever l'exception si un mot n'est pas trouvé dans les embeddings
        embedding_vector = embeddings.wv[word] 
        embedding_matrix[i] = embedding_vector
    except KeyError: # mot non trouvé dans les embeddings
        pass
  return embedding_matrix

DIMENSION=100

embedding_matrix=get_embedding_matrix(model_stanford,mytokens,vocab_size,DIMENSION)

Dans la cellule suivante, nous proposons un premier modèle de classification. Nous pouvons constater que pour pouvoir utiliser des embeddings existants, contrairement à l'apprentissage d'embeddings où les poids sont définis de manière aléatoire, il est nécessaire de les passer en paramètres dans la couche : 
*Embedding(vocab_size, dimension, weights=[embedding_matrix], input_length=max_length, trainable=False)*

In [ ]:
from IPython.display import Image 
from keras.utils.vis_utils import plot_model

def MyfirstModel(vocab_size, max_length, embedding_matrix, dimension):
  model = Sequential()
  model.add(Embedding(vocab_size, dimension, weights=[embedding_matrix], input_length=max_length, trainable=False)) 
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  # compilation du réseau - adam pour accélérer
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
  # résumé du réseau
  model.summary()
  #plot_model(model, to_file='model.png', show_shapes=True)
  plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
  Image('model.png')
  return model


Le jeu de données est vraiment petit. Malgré cela, nous séparons le jeu de données d'apprentissage en jeu d'apprentissage et de tests pour le modèle. 
L'appel se fait alors par : 

In [ ]:
# découpage du jeu d'apprentissage en apprentissage et test pour évaluer le modèle en apprentissage
seed=7
X_train, X_val, y_train, y_val = train_test_split(padded_docs, labels, 
                                                              test_size=0.3, random_state=seed)
firstmodel = MyfirstModel(vocab_size, MAX_LENGTH, embedding_matrix, DIMENSION)
history=firstmodel.fit(X_train, y_train, epochs=20, verbose=2,validation_data=(X_val, y_val))

Affichage de l'historique

In [ ]:
def plot_history(history):
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('accuracy ('+str(format(history.history['accuracy'][-1],'.3f'))+')')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('loss ('+str(format(history.history['loss'][-1],'.3f'))+')')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

plot_history(history) 

L'objectif maintenant est de voir si le modèle appris permet de bien classer.   
Pour cela nous ajoutons des données qui n'ont pas été apprises. Il est indispensable pour pouvoir prédire la classe de transformer les données comme précédemment (tokens, padding, même longueur de vecteur, ...).

In [ ]:

# Utilisation d'un nouveau jeu de tests 
testmodel=["Mice are gray",
          "The beautiful red tomatoes",
          "The dog runs in the forest"]

testlabel= array([1,1,0])      

# comme précédemment il est nécessaire de transformer en tokens
mytesttokens = Tokenizer()
mytesttokens.fit_on_texts(mesdocs)
print ("affichage premiers tokens",mytesttokens.word_index)

# la taille du vocabulaire doit donc toujours être supérieure à 1 (d'où le +1)
vocab_size_test = len(mytesttokens.word_index) + 1
print ("vocab_size",vocab_size_test)

# transformation en séquences et padding
# la méthode texts_to_sequences permet de transformer les documents avec l'index associé
encoded_docs_test = mytokens.texts_to_sequences(testmodel)
print ("docs",testmodel)
print("encoded_docs",encoded_docs_test)
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=MAX_LENGTH, padding='post')
print("padded_docs",padded_docs_test)

La prédiction se fait alors par : 

In [ ]:

y_pred=(firstmodel.predict(padded_docs_test) > 0.5).astype("int32")

# Affichage des résultats
MyshowAllScores(testlabel,y_pred)


La cellule suivante regroupe en une seule fois toutes les cellules précédentes afin d'avoir une vision globale avec l'approche classique jeu de données et jeu d'apprentissage : 

In [ ]:
mesdocs = ["The sky is really blue today",
           "Bananas are normally yellow",
          "The nice dog jumps in the air",
          "The forest is all green",
           "The cat is purring",
           "The car drives quietly on the road",
            "The red of this car is beautiful",
           "Embeddings can help classify better",
           "Mice are gray",
           "The beautiful red tomatoes",
          "The dog runs in the forest"
           ]

# les labels des classes sont positionnés à 1 si le mot contient une couleur
labels = array([1,1,0,1,0,0,1,0,1,1,0])

# separation du jeu de données en test et apprentissage
X_train, X_test, y_train, y_test = train_test_split(mesdocs, labels, test_size=0.2, random_state=seed)

seed=7
MAX_LENGTH = 20
DIMENSION=100
# découpage en jeu de test et d'apprentissage
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# transformation du jeu d'apprentissage
tokens = Tokenizer()
tokens.fit_on_texts(X_train)
vocab_size = len(tokens.word_index) + 1
encoded_docs = tokens.texts_to_sequences(X_train)
X_train = pad_sequences(encoded_docs, maxlen=MAX_LENGTH, padding='post')
# calcul de la matrice des embeddings
embedding_matrix=get_embedding_matrix(model_stanford,tokens,vocab_size,DIMENSION)

# Création du modèle
firstmodel =MyfirstModel(vocab_size, MAX_LENGTH, embedding_matrix, DIMENSION)

# découpage du jeu d'apprentissage en apprentissage et test pour évaluer le modèle en apprentissage
X_train_transformed, X_val_transformed, y_train, y_val = train_test_split(X_train, y_train, 
                                                              test_size=0.2, random_state=seed)
# application du modèle
history=firstmodel.fit(X_train_transformed, y_train, epochs=50, verbose=2, 
                       validation_data=(X_val_transformed,y_val))

plot_history(history)

# prediction 
# transformation du jeu de test
testtokens = Tokenizer()
testtokens.fit_on_texts(X_test)
vocab_size_test = len(testtokens.word_index) + 1
encoded_docs_test = testtokens.texts_to_sequences(X_test)
X_test_transformed = pad_sequences(encoded_docs_test, maxlen=MAX_LENGTH, padding='post')

# calcul de la prédiction 
y_pred=(firstmodel.predict(X_test_transformed) > 0.5).astype("int32")

# Affichage des résultats
MyshowAllScores(y_test,y_pred)


<font color=red>Exercice :</font> compléter la cellule suivante pour créer un nouveau classifier sur le jeu de données ReviewsLabelled.csv. Nous restons sur une classification binaire (à noter la fonction sigmoid qui devrait être remplacée par une fonction softmax à la fin du réseau pour une classification multiclasses). 

In [ ]:
def MyfirstModel(vocab_size, max_length, embedding_matrix, dimension):
  model = Sequential()
  model.add(Embedding(vocab_size, dimension, weights=[embedding_matrix], input_length=max_length, trainable=False)) 
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  # compilation du réseau - adam pour accélérer
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
  # résumé du réseau
  model.summary()
  #plot_model(model, to_file='model.png', show_shapes=True)
  plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
  Image('model.png')
  return model



df_donnees = pd.read_csv("ReviewsLabelled.csv", names=['sentence','sentiment','source'], header=0,sep='\t', encoding='utf8')


print ("mélange du fichier ")
df_donnees=df_donnees.sample(frac=1).reset_index(drop=True)

print ("les 10 premières lignes du fichier :")
display(df_donnees[0:10])

# selection des données
X=df_donnees.sentence
y=df_donnees.sentiment




<font color=blue>Solution :</font>

In [ ]:
def MyfirstModel(vocab_size, max_length, embedding_matrix, dimension):
  model = Sequential()
  model.add(Embedding(vocab_size, dimension, weights=[embedding_matrix], input_length=max_length, trainable=False)) 
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  # compilation du réseau - adam pour accélérer
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
  # résumé du réseau
  model.summary()
  #plot_model(model, to_file='model.png', show_shapes=True)
  plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
  Image('model.png')
  return model



df_donnees = pd.read_csv("ReviewsLabelled.csv", names=['sentence','sentiment','source'], header=0,sep='	', encoding='utf8')


df_donnees=df_donnees.sample(frac=1).reset_index(drop=True)
print(df_donnees[0:10])

# selection des données
X=df_donnees.sentence
y=df_donnees.sentiment

seed=7
MAX_LENGTH = 20
DIMENSION=100

# découpage en jeu de test et d'apprentissage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)


# traitement des données d'apprentissage
tokens = Tokenizer()
tokens.fit_on_texts(X_train)
print ("affichage premiers tokens",tokens.word_index)
vocab_size = len(tokens.word_index) + 1
print ("vocab_size",vocab_size)

# pour pouvoir traiter les documents, il est nécessaire de les encoder en entier 
# la méthode texts_to_sequences permet de transformer les documents avec l'index associé
encoded_docs = tokens.texts_to_sequences(X_train)

MAX_LENGTH = 100
X_train_transformed = pad_sequences(encoded_docs, maxlen=MAX_LENGTH, padding='post')


embedding_matrix=get_embedding_matrix(model_stanford,tokens,vocab_size,DIMENSION)
# découpage du jeu d'apprentissage en apprentissage et test pour évaluer le modèle en apprentissage
X_train_transformed, X_val_transformed, y_train, y_val = train_test_split(X_train_transformed, y_train, 
                                                              test_size=0.2, random_state=seed)

# creation du modèle
firstmodel = MyfirstModel(vocab_size, MAX_LENGTH, embedding_matrix, DIMENSION)


history=firstmodel.fit(X_train_transformed, y_train, epochs=50, verbose=2, 
                       validation_data=(X_val_transformed,y_val))

plot_history(history)
# calcul de la prédiction 
testtokens = Tokenizer()
testtokens.fit_on_texts(X_test)
vocab_size_test = len(testtokens.word_index) + 1
encoded_docs_test = testtokens.texts_to_sequences(X_test)
X_test_transformed = pad_sequences(encoded_docs_test, maxlen=MAX_LENGTH, padding='post')

y_pred=(firstmodel.predict(X_test_transformed) > 0.5).astype("int32")

# Affichage des résultats
MyshowAllScores(y_test,y_pred)

<font color=red>Exercice :</font> tester le code précédent mais en utilisant les deux modèles définis ci-dessous. Le premier utilise des CNN et le second du LSTM, i.e. il tient compte de l'ordre des mots. Que constatez vous par rapport aux autres classifiers. 

In [ ]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

def MyCNNModel(vocab_size, max_length, embedding_matrix, dimension):
  model = Sequential()
  model.add(Embedding(vocab_size, dimension, weights=[embedding_matrix], input_length=max_length, trainable=False)) 
  model.add(Conv1D(filters=32, kernel_size=8, activation='relu')) 
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # compile network
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
  model.summary()
  return model

def MyLSTMModel(vocab_size, max_length, embedding_matrix, dimension):
  model = Sequential()
  model.add(Embedding(vocab_size, dimension, weights=[embedding_matrix], input_length=max_length, trainable=False))
  model.add(Dropout(0.5))
  model.add(Bidirectional(LSTM(dimension)))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
  model.summary()
  return model

<font color=blue>Solution :</font>

In [ ]:
# creation du modèle
print ("Utilisation de CNN")
firstmodel = MyCNNModel(vocab_size, MAX_LENGTH, embedding_matrix, DIMENSION)


history=firstmodel.fit(X_train_transformed, y_train, epochs=20, verbose=2, 
                       validation_data=(X_val_transformed,y_val))

plot_history(history)
# calcul de la prédiction 
y_pred=(firstmodel.predict(X_test_transformed) > 0.5).astype("int32")

# Affichage des résultats
MyshowAllScores(y_test,y_pred)

print ("Utilisation de LSTM")
secondmodel = MyLSTMModel(vocab_size, MAX_LENGTH, embedding_matrix, DIMENSION)


history=secondmodel.fit(X_train_transformed, y_train, epochs=20, verbose=2, 
                       validation_data=(X_val_transformed,y_val))

plot_history(history)
# calcul de la prédiction 
y_pred=(secondmodel.predict(X_test_transformed) > 0.5).astype("int32")

# Affichage des résultats
MyshowAllScores(y_test,y_pred)

Le fait d'utiliser des word embeddings pour la classification de textes est un réel avantage. Il est courant, voir indispensable maintenant, lorsque l'on travaille sur des données (e.g. compte rendu médical, relevés d'opérations, comptes rendus, documents dans une langue étrangère, etc.) pas trop classiques comme les dépêches de presse ou des opinions, pour lesquelles il existe déjà des embeddings, de pouvoir apprendre le plus de contexte possible et donc avant de faire une classification d'apprendre sur l'ensemble du corpus.
